In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.interpolate import griddata
from PIL import Image
from tqdm import tqdm
from masks import masks_original_dim
import cv2


resized_dim = (300, 300)

# list the paths to the folder containing .png files
high_res_greyscale_image_folders = [r'.\images 15000x15000\SAC105',
	r'.\images 15000x15000\SAC305',
	r'.\images 15000x15000\Senju',
]

# list the paths to the folder containing .npy files corresponding to previous list elements
folders_with_npy_files = [r'.\..\OneDrive_2025-03-03\LED_data_for prediction\Dataset\Data_preprocessing_2D_CNN\Data_All\Data\SAC105',
	r'.\..\OneDrive_2025-03-03\LED_data_for prediction\Dataset\Data_preprocessing_2D_CNN\Data_All\Data\SAC305',
	r'.\..\OneDrive_2025-03-03\LED_data_for prediction\Dataset\Data_preprocessing_2D_CNN\Data_All\Data\Senju',
]

# path to the folder to save the images
masked_images_folder_path = r'.\images\masked_images'








for high_res_greyscale_image_folder in high_res_greyscale_image_folders:
	# Create the directory if it doesn't exist to save the images
	os.makedirs(os.path.join(masked_images_folder_path, os.path.basename(high_res_greyscale_image_folder)), exist_ok=True)

	# Get a list of all .png files in the folder
	png_files_path = [os.path.join(high_res_greyscale_image_folder, file) for file in os.listdir(high_res_greyscale_image_folder) if file.endswith('.png')]


	for png_file_path in tqdm(png_files_path):
		# get the basename for the low res greyscale image and also name of pad design (basename of png_file_path = grayscale_FCGB1_sac105_15000x15000.png)
		basename = 'masked_' + os.path.basename(png_file_path)[:-4]  # (basename = masked_grayscale_FCGB1_sac105_15000x15000)
		basename = '_'.join(basename.split('_')[:-1])  # (basename = masked_grayscale_FCGB1_sac105)
		pad_design_name = basename.split('_')[-2]  # (pad_design_name = FCGB1)
		basename = basename + f'_{resized_dim[0]}x{resized_dim[1]}.png'  # (basename = masked_grayscale_FCGB1_sac105_300x300.png)

		# define the path to save the low res greyscale image and masked low res greyscale image
		low_res_greyscale_image_save_path = os.path.join(masked_images_folder_path, os.path.basename(high_res_greyscale_image_folder), basename.replace('masked_', ''))
		masked_low_res_greyscale_image_save_path = os.path.join(masked_images_folder_path, os.path.basename(high_res_greyscale_image_folder), basename)


		# Load the image and resize it
		image = cv2.imread(png_file_path, cv2.IMREAD_GRAYSCALE)  # Load as greyscale image	
		resized_image = cv2.resize(image, resized_dim)

		# save the resized image as a low res greyscale image
		cv2.imwrite(low_res_greyscale_image_save_path, resized_image)
		absolute_path = os.path.abspath(low_res_greyscale_image_save_path)
		# print(f"The masked low res image was saved to: {absolute_path}")

		# # Plot the resized image using matplotlib
		# plt.imshow(resized_image, cmap='gray', vmin=0, vmax=255)
		# plt.title(f"Resized Image {resized_dim}")
		# plt.colorbar(label='Pixel Intensity')
		# plt.axis('on')  # Show the axis
		# plt.gca().invert_yaxis()  # Invert the y-axis
		# plt.show()


		# get data from masks_original_dim dictionary using pad_design_name
		(data_x_min, data_x_max, data_y_min, data_y_max), *rectangles = masks_original_dim[pad_design_name]

	    # find the scale in each axis
		x_range, y_range = data_x_max - data_x_min, data_y_max - data_y_min
		x_scale = resized_dim[0] / x_range
		y_scale = resized_dim[1] / y_range

		# Loop through the original rectangles and scale them
		for (x1, y1, x2, y2) in rectangles:
			# print(resized_image.shape, (x1, x2, y1, y2))
			# Move the origin of the rectangle to the top-left
			x1 -= data_x_min
			x2 -= data_x_min
			y1 -= data_y_min
			y2 -= data_y_min

			# Scale the dimensions to resized_dim
			scaled_x1 = x1 * x_scale
			scaled_x2 = x2 * x_scale
			scaled_y1 = y1 * y_scale
			scaled_y2 = y2 * y_scale

			# Ensure valid rectangle dimensions
			scaled_x1, scaled_x2 = sorted([scaled_x1, scaled_x2])
			scaled_y1, scaled_y2 = sorted([scaled_y1, scaled_y2])

			# Overwrite the pixels in the rectangle region with black
			resized_image[int(scaled_y1):int(scaled_y2), int(scaled_x1):int(scaled_x2)] = 0

			# print(resized_image.shape, (scaled_x1, scaled_y1, scaled_x2, scaled_y2))
		
		
		# Save the image and print the message
		cv2.imwrite(masked_low_res_greyscale_image_save_path, resized_image)
		absolute_path = os.path.abspath(masked_low_res_greyscale_image_save_path)
		# print(f"The masked low res image was saved to: {absolute_path}")

		# Optionally, display the image
		# cv2.imshow('Masked image', resized_image)
		cv2.waitKey(0)
		cv2.destroyAllWindows()


100%|██████████| 7/7 [00:09<00:00,  1.31s/it]
